In [1]:
import glob
import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import pandas as pd
data_dir = 'data_meg'
subj = "R2210"
dtype = "raw"
raw = mne.io.read_raw_fif('data_meg/R2210_raw_ica.fif', preload=True)
save_dir = 'data_meg'


Opening raw data file data_meg/R2210_raw_ica.fif...


/var/folders/4s/v49tr3_d3vn5jcm7fg7z3cv00000gp/T/ipykernel_24427/2296937516.py:12: RuntimeWarning: This filename (data_meg/R2210_raw_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif('data_meg/R2210_raw_ica.fif', preload=True)


    Range : 0 ... 2779999 =      0.000 ...  2779.999 secs
Ready.
Opening raw data file /Users/labadmin/meg_decoding/data_meg/R2210_raw_ica-1.fif...
    Range : 2780000 ... 2954999 =   2780.000 ...  2954.999 secs
Ready.
Reading 0 ... 2954999  =      0.000 ...  2954.999 secs...


In [2]:
labels_df = pd.read_csv('label.csv')

In [3]:
sfreq = raw.info['sfreq']
raw.filter(1, 40, method='iir')
downsample =10 

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB



In [4]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 1,
    'move': 2,
    'reveal_red': 4,
    'reveal_white': 8,
    'done': 16,
}


5438 events found on stim channel STI 014
Event IDs: [ 1  2  4  8 16]


In [5]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
sfreq = raw.info['sfreq']  # Sampling frequency

# Initialize a list to store trial information
trial_info = []

done_idx = 0

for start_event in start_events:
    start_sample = start_event[0]
    
    # Find the corresponding 'done' event
    while done_idx < len(done_events) and done_events[done_idx, 0] <= start_sample:
        done_idx += 1
    if done_idx < len(done_events):
        done_sample = done_events[done_idx, 0]
        

        # Calculate tmin and tmax for the epoch
        tmin = -0.2  # 0.2 s before 'start'
        tmax = (done_sample - start_sample) / sfreq + 1.0  # Duration from 'start' to 1 s after 'done'

        # Store trial information
        trial_info.append({
            'event_sample': start_sample,
            'trial_index': done_idx,
            'duration': tmax,
            'tmin': tmin,
            'tmax': tmax
        })
        done_idx += 1
max_tmax = max([info['tmax'] for info in trial_info])
print(f"Maximum epoch duration: {max_tmax:.2f} seconds")


Maximum epoch duration: 78.10 seconds


In [6]:
new_events = np.array([[info['event_sample'], 0, event_id['start']] for info in trial_info])
tmin = -0.2  
# Initialize lists to store individual epochs data and trial information
epochs_data_list = []
trial_info_valid = []

In [7]:
print(raw.times[-1])

2954.999


In [8]:
for idx, event in enumerate(new_events):
    start_sample = event[0]
    event_id_code = event[2]
    event_time = start_sample / sfreq
    total_duration = raw.times[-1]
    
    # Determine available duration based on the next event or end of recording
    if idx < len(new_events) - 1:
        end_sample = new_events[idx + 1][0]
    else:
        end_sample = raw.n_times  
    
    # Calculate the available duration
    available_duration = (end_sample - start_sample) / sfreq
    print(available_duration)
    # Adjust tmin and baseline for the last epoch
    if idx == len(new_events) - 1:
        # Adjust tmin if baseline period is invalid
        if (start_sample + tmin * sfreq) < 0:
            tmin_adjusted = - event_time + 0.001  # Small buffer
            baseline = None  # Disable baseline correction
            print(f"Adjusting tmin for last epoch to {tmin_adjusted} seconds and disabling baseline correction.")
        else:
            tmin_adjusted = tmin
            baseline = (tmin_adjusted, 0)
        # Adjust tmax based on available data
        max_possible_tmax = available_duration + tmin_adjusted
        actual_tmax = min(max_tmax, max_possible_tmax)
        print(f"Adjusting tmax for last epoch to {actual_tmax} seconds.")
    else:
        tmin_adjusted = tmin
        baseline = (tmin_adjusted, 0)
        actual_tmax = min(max_tmax,available_duration - tmin)
        print(f"Tmax for {actual_tmax} seconds")
    
    # Calculate epoch duration
    epoch_duration = actual_tmax - tmin_adjusted
    
    # Ensure epoch duration is positive and sufficient
    min_epoch_duration = 1 / sfreq  # Minimum duration is one sample
    
    # Create the epoch
    epochs = mne.Epochs(
        raw, [event], event_id={f'event_{event_id_code}': event_id_code},
        tmin=tmin_adjusted, tmax=actual_tmax, baseline=baseline, preload=True,
        reject_by_annotation=False, reject=None, verbose=True
    )
    
    if len(epochs) > 0:
        epochs_data_list.append(epochs.get_data()[:, :, ::downsample])
        trial_info_valid.append(trial_info[idx])
    else:
        print(f"Epoch {idx} was dropped.")
        print(f"Drop log for Epoch {idx}: {epochs.drop_log}")


29.818
Tmax for 30.018 seconds
Not setting metadata
1 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 30219 original time points ...
0 bad epochs dropped
39.18
Tmax for 39.38 seconds
Not setting metadata
1 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 39581 original time points ...
0 bad epochs dropped
20.376
Tmax for 20.576 seconds
Not setting metadata
1 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 20777 original time points ...
0 bad epochs dropped
56.717
Tmax for 56.917 seconds
Not setting metadata
1 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 57118 original time points ...
0 bad epochs dropped
22.825
Tmax for 23

In [9]:
# Create a unified masked data structure with variable lengths
max_channels = raw.info['nchan']
masked_data_list = []

for data, info in zip(epochs_data_list, trial_info_valid):
    # Extract shape
    n_channels, n_times = data.shape[1], data.shape[2]

    # Create a valid data mask
    valid_data_mask = np.zeros((n_channels, n_times), dtype=bool)
    duration_samples = int((info['tmax'] - tmin) * sfreq)
    valid_data_mask[:, :duration_samples] = True

    # Create masked array for the epoch
    masked_epoch_data = ma.masked_array(data[0], mask=~valid_data_mask)
    masked_data_list.append(masked_epoch_data)

# Print information about the created epochs
print(f"Number of trials in trial_info_valid: {len(trial_info_valid)}")
print(f"Number of epochs after handling short recordings: {len(masked_data_list)}")

# Example usage: iterate over masked data for further analysis
for idx, masked_data in enumerate(masked_data_list):
    print(f"Epoch {idx + 1}: Shape = {masked_data.shape}, Masked = {masked_data.mask.sum()} points")

Number of trials in trial_info_valid: 120
Number of epochs after handling short recordings: 120
Epoch 1: Shape = (193, 3022), Masked = 0 points
Epoch 2: Shape = (193, 3959), Masked = 0 points
Epoch 3: Shape = (193, 2078), Masked = 0 points
Epoch 4: Shape = (193, 5712), Masked = 0 points
Epoch 5: Shape = (193, 2323), Masked = 0 points
Epoch 6: Shape = (193, 2312), Masked = 0 points
Epoch 7: Shape = (193, 3096), Masked = 0 points
Epoch 8: Shape = (193, 1205), Masked = 0 points
Epoch 9: Shape = (193, 7831), Masked = 0 points
Epoch 10: Shape = (193, 7831), Masked = 0 points
Epoch 11: Shape = (193, 2733), Masked = 0 points
Epoch 12: Shape = (193, 3366), Masked = 0 points
Epoch 13: Shape = (193, 4168), Masked = 0 points
Epoch 14: Shape = (193, 1701), Masked = 0 points
Epoch 15: Shape = (193, 1575), Masked = 0 points
Epoch 16: Shape = (193, 3284), Masked = 0 points
Epoch 17: Shape = (193, 3226), Masked = 0 points
Epoch 18: Shape = (193, 3109), Masked = 0 points
Epoch 19: Shape = (193, 1127), 

In [10]:
# Step 1: Determine the maximum length across all epochs
max_times = max([masked_data.shape[2] for masked_data in epochs_data_list])

# Step 2: Create arrays to store padded epoch data and corresponding masks
n_epochs = len(epochs_data_list)
n_channels = raw.info['nchan']

# Initialize the padded epochs data array with NaNs
epochs_array = np.full((n_epochs, n_channels, max_times), np.nan)

# Initialize the mask array with False (will set to True for padded values)
mask_array = np.zeros((n_epochs, n_channels, max_times), dtype=bool)

# Step 3: Fill the data and create the mask for each epoch
for idx, (data, info) in enumerate(zip(epochs_data_list, trial_info_valid)):
    n_times = data.shape[2]
    # Fill the epochs array with the actual data
    epochs_array[idx, :, :n_times] = data[0]
    
    # Set the mask to True for the padded values
    mask_array[idx, :, n_times:] = True

# Step 4: Create a masked array using the data and the mask
masked_data_array = ma.masked_array(epochs_array, mask=mask_array)



## Decoding

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import make_pipeline
from mne.decoding import SlidingEstimator, cross_val_multiscore 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from matplotlib import pyplot as plt
label_encoder = LabelEncoder()

In [12]:
n_labels = len(labels_df)
n_trials = len(trial_info_valid)
print(f"Number of labels: {n_labels}")
print(f"Number of valid trials: {n_trials}")
# Create a mapping from trial_index to label
labels_df['trial_index'] = labels_df['trial_index'].astype(int)
label_dict = dict(zip(labels_df['trial_index'], labels_df['trial.rule']))

# Extract labels for the valid trials
y_labels = []
for info in trial_info_valid:
    idx = info['trial_index']
    y_labels.append(label_dict[idx])

# Convert labels to integers
y = label_encoder.fit_transform(y_labels)


Number of labels: 120
Number of valid trials: 120


In [13]:

# Determine the target length (e.g., the maximum length)
target_length = max([md.shape[1] for md in masked_data_list])

# Process data
processed_data_list = []
for masked_epoch_data in masked_data_list:
    n_channels, n_times = masked_epoch_data.shape
    data = masked_epoch_data.data
    mask = masked_epoch_data.mask

    # Initialize arrays with the target length
    padded_data = np.zeros((n_channels, target_length))
    padded_mask = np.ones((n_channels, target_length), dtype=bool)  # Start with all masked

    # Copy valid data
    valid_length = min(n_times, target_length)
    padded_data[:, :valid_length] = data[:, :valid_length]
    padded_mask[:, :valid_length] = mask[:, :valid_length]

    # Create a new masked array
    processed_masked_epoch = np.ma.masked_array(padded_data, mask=padded_mask)
    processed_data_list.append(processed_masked_epoch)


In [14]:
n_epochs = len(processed_data_list)
n_channels = processed_data_list[0].shape[0]

# Stack data into an array
X = np.array([md.data.flatten() for md in processed_data_list])  # Shape: (n_epochs, n_channels * target_length)
mask_array = np.array([md.mask.flatten() for md in processed_data_list])

# Handle masked values
# Replace masked values with zeros
X_imputed = np.where(mask_array, 0, X)

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Cross-validation

In [16]:
# Define the classifier
clf = LogisticRegression(max_iter=1000)

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate the classifier using cross-validation
scores = cross_val_score(clf, X_scaled, y, cv=cv, scoring='accuracy')

# Print the results
print(f"Cross-validation accuracy scores: {scores}")
print(f"Mean accuracy: {scores.mean() * 100:.2f}%")

Cross-validation accuracy scores: [0.41666667 0.33333333 0.5        0.375      0.58333333]
Mean accuracy: 44.17%


In [17]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report

# Get cross-validated predictions
y_pred = cross_val_predict(clf, X_scaled, y, cv=cv)

# Compute confusion matrix
cm = confusion_matrix(y, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification report
report = classification_report(y, y_pred, target_names=label_encoder.classes_)
print("Classification Report:")
print(report)


Confusion Matrix:
[[23 13  2]
 [21 12  6]
 [16  9 18]]
Classification Report:
              precision    recall  f1-score   support

       chain       0.38      0.61      0.47        38
        loop       0.35      0.31      0.33        39
        tree       0.69      0.42      0.52        43

    accuracy                           0.44       120
   macro avg       0.48      0.44      0.44       120
weighted avg       0.48      0.44      0.44       120



# Decoding Learning 